# TOC

__Lab 07 - Classification__
1. [Import](#Import)
1. [Multi-class classification](#Multi-class-classification)
    1. [Example 1 - Distance from planes](#Distance-from-planes-Example1)
    1. [Example 2 - Iris classification w/ logistic regression](#Iris-classification-w/-logistic-regression-Example2)


# Import

<a id = "Import"></a>

In [52]:
import numpy as np; np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:78% !important; }</style>"))

import os
import itertools 

from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import load_iris, fetch_mldata
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

sns.set(style = 'whitegrid', font_scale = 1.3)
matplotlib.rcParams['legend.framealpha'] = 1
matplotlib.rcParams['legend.frameon'] = True


# Multi-class classification

Tne method for classifying an observation in a multi-class problem is to calculate the perpendicular distance of the observation from each of the separating planes. The shortest distance is the class selection.

<a id = "Multi-class-classification"></a>

## Example 1 - Distance from planes
Consider a three-class classification problem with the following separating hyperplanes:

$$
\begin{align}
x_1 + x_2 - 2 = 0 & & 1^{\text{st}} \text{ class}\\
-4x_1 + 3x_2 - 11 = 0 & & 2^{\text{nd}} \text{ class}\\
-x_1 - 3x_2 - 1 = 0 & & 3^{\text{rd}} \text{ class}
\end{align}
$$

Calculate the score (unnormalized perpendicular distance) for each class for the test case $(x_1, x_2)=(0, 1)$ .


<a id = "Distance-from-planes-Example1"></a>

In [ ]:
# plot planes and observation
x = np.arange(-3, 5)
plt.plot(x, (2. - 1 * x) / 1., label = '1st', lw = 5)
plt.plot(x, (11. + 4 * x) / 3., label = '2nd', lw = 5)
plt.plot(x, (-1. - 1 * x) / 3., label = '3rd', lw = 5)

plt.gca().set_aspect('equal', adjustable = 'box')
plt.scatter([0],[1], c = 'k', s = 100)
plt.xlim([-3, 4])
plt.ylim([-3, 4])
plt.grid()
plt.legend()
plt.show()


In [ ]:
# build arrays for weights, bias units and observations
W = np.transpose(np.array([
    [1., 1.]
    ,[-4, 3.]
    ,[-1., -3.]
    ])).reshape(2, 3)
b = np.array([-2.
             ,-11.
             ,-1.])

# test point
x = np.array([0., 1.])


In [13]:
# review data
print(W)
print(b)
print(x)


[[ 1. -4. -1.]
 [ 1.  3. -3.]]
[ -2. -11.  -1.]
[0. 1.]


In [ ]:
# determine distance from each plane
def scores (W, b, x):
    return np.dot(x, W) + b
scores(W, b, x)


> Observations is closest to class 1

In [ ]:
# determine probabilities associated with predictions
def probas(W, b, x):
    sc = scores(W, b, x)
    scExp = np.exp(sc)
    return scExp / np.sum(scExp)

np.round(probas(W, b, x), 5)


> .95 probability that observatation belongs to class one

In [ ]:
# generate the answer
def predictClass(W, b, x):
    prob = probas(W, b, x)
    return np.argmax(prob) + 1

predictClass(W, b, x)


> Class 1 is chosen

## Example 2 - Iris classification w/ logistic regression
Build a classifier to identify an iris sample as belonging to one of three iris classes based on four features.

<a id = "Iris-classification-w/-logistic-regression-Example2"></a>

In [ ]:
# load data
iris = load_iris()


In [20]:
# create observation and label datasets
X = pd.DataFrame(iris.data, columns = iris.feature_names)
y = iris.target


In [ ]:
# nspect features
X.head()


In [30]:
# inspect feature summary statistics
X.describe()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


>All the features here are continuous numerical features

__EDA__

In [ ]:
# correlation matrix
classLabels = ['setosa', 'versicolour', 'virginica']

corr = X.corr()
mask = np.zeros_like(corr, dtype = bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize = (20,20))
cmap = sns.diverging_palette(220, 10, as_cmap = True)
sns.heatmap(corr, mask = mask, cmap = cmap, vmin = -1.0, vmax = 1.0, square = True
           ,linewidths = .5, cbar_kws = {'shrink' : .5}, ax = ax)


>Petal width and petal length are strongly positively correlated. Petal length and width are somewhat negatively correlated with sepal width.

In [ ]:
# pair-wise plots
sns.pairplot(sns.load_dataset('iris'), hue = 'species', size = 3)


>The correlations describes above are apparent. Additionaly the setosa class (identified by blue dots) is often completely separated from the versicolor and virginica classes.

In [ ]:
# train/test split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [ ]:
# fitting
logReg = LogisticRegression(C = 1e6, multi_class = 'multinomial', solver = 'lbfgs')
logReg.fit(xTrain, yTrain)


In [ ]:
# model evaluation
plt.figure(figsize = (12, 8))
plt.bar(np.arange(logReg.coef_.shape[1]) - 0.2, logReg.coef_[0], color = 'red', width = 0.2, label = 'setosa')
plt.bar(np.arange(logReg.coef_.shape[1]) - 0.0, logReg.coef_[1], color = 'green', width = 0.2, label = 'versicolour')
plt.bar(np.arange(logReg.coef_.shape[1]) + 0.2, logReg.coef_[2], color = 'blue', width = 0.2, label = 'virginica')
plt.xticks(np.arange(logReg.coef_.shape[1]), X.columns, rotation = 90)
plt.xlim(-1, logReg.coef_.shape[1])
plt.title('Model coefficients')
plt.legend(loc = 'lower right')


>The most pronounced features in terms of the coefficients are pedal length and pedal width. This is consistent with then pairwise plots above in the these features seem to have clear divisions between each class.

In [ ]:
# test set predictions
yPredsTrain = logReg.predict(xTrain)
yPredsTest = logReg.predict(xTest)


In [41]:
# review accuracy
print('Train accuracy: ', np.round(accuracy_score(yTrain, yPredsTrain), 3))
print('Test accuracy: ', np.round(accuracy_score(yTest, yPredsTest), 3))


Train accuracy:  0.983
Test accuracy:  1.0


In [ ]:
# log results
results = pd.DataFrame(columns = ['Accuracy'])
results = results.append(pd.DataFrame(
        [np.round(accuracy_score(yTrain, yPredsTrain), 3), np.round(accuracy_score(yTest, yPredsTest), 3)], 
        columns=['Accuracy'], 
        index=['Sklearn Train', 'Sklearn Test']))
results

__Custom function__

In [38]:
# custom function
def plot_confusion_matrix(cm
                          ,classes
                          ,normalize = False
                          ,title = 'Confusion matrix'
                          ,cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment = 'center',
                 color = 'white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

In [ ]:
# confusion matrix
np.set_printoptions(precision = 2)

# plot normalized confusion matrix
plt.figure(figsize = (10,10))
plot_confusion_matrix(cm = confusion_matrix(yTest, yPredsTest)
                      ,classes = classLabels
                      ,normalize = True
                      ,title = 'Normalized confusion matrix - Test set')

plt.show()


> The test class is perfectly classified